In [25]:
import netket as nk
import netket.experimental as nkx

L_x = 2
L_y = 2
N_up = N_down = 2
pbc = False

if N_up is None:
    hilb8 = nk.hilbert.SpinOrbitalFermions(
        n_orbitals=L_x * L_y, s=1/2# , n_fermions_per_spin=(N_up, N_down)
    )
else:
    hilb8 = nk.hilbert.SpinOrbitalFermions(
        n_orbitals=L_x * L_y, s=1/2, n_fermions_per_spin=(N_up, N_down)
    )

hi = hilb8

t = 1
U = 8


def c(site, sz):
    return nk.operator.fermion.destroy(hi, site, sz=sz)


def cdag(site, sz):
    return nk.operator.fermion.create(hi, site, sz=sz)


def nc(site, sz):
    return nk.operator.fermion.number(hi, site, sz=sz)


g = nk.graph.Grid(extent=[L_x, L_y], pbc=pbc)
g8 = g
up = +1
down = -1
ham = 0.0

for sz in (up, down):
    for u, v in g.edges():
        # Map tile-local indices to global indices
        ham += -t * cdag(u, sz) @ c(v, sz) - t * cdag(v, sz) @c(u, sz)
for u in g.nodes():
    # Only add Hubbard U term for sites inside tile 0
    ham += U * nc(u, up) @ nc(u, down)


H8 = ham # nkx.operator.ParticleNumberAndSpinConservingFermioperator2nd.from_fermionoperator2nd(ham)

eig_vals, eig_vecs = nk.exact.lanczos_ed(H8, k=1, compute_eigenvectors=True)
print(f"Ground state energy: {eig_vals[0]}")





import numpy as np
from openfermion import fermi_hubbard, jordan_wigner, QubitOperator
from scipy.sparse.linalg import expm_multiply
from scipy.sparse import csc_matrix
from openfermion.linalg import get_sparse_operator
from scipy.sparse.linalg import eigsh

# -----------------------------
# 1. Hubbard Hamiltonian
# -----------------------------
# L_x, L_y = 3, 3
# N_up = N_down = 2
n_orbitals = L_x * L_y
n_qubits = 2 * n_orbitals  # Spin up and down
t = 1.0
U = 8.0

hubbard = fermi_hubbard(L_x, L_y, tunneling=-t, coulomb=U, periodic=pbc)
qubit_op = jordan_wigner(hubbard)  # QubitOperator

H_sparse = csc_matrix(get_sparse_operator(qubit_op, n_qubits=n_qubits))

# Niedrigste Eigenwerte von H_sparse (vollständige Spektrumsberechnung ist zu groß)
k = 1
eigs, eigv = eigsh(H_sparse, k=k, which='SA', return_eigenvectors=True)
if N_up is not None:
    warning = "N_up, N_down not supported, without particle number conservation"
else:
    warning = ""
print("Lowest eigenvalues:", eigs, warning)

def count_up_down(bitstring, n_orbitals):
    n_up = 0
    n_dn = 0
    for site in range(n_orbitals):
        up_bit = (bitstring >> (2*site)) & 1
        dn_bit = (bitstring >> (2*site + 1)) & 1
        n_up += up_bit
        n_dn += dn_bit
    return n_up, n_dn

full_dim = 2**n_qubits
if N_up is not None:
    valid_indices = []
    for i in range(full_dim):
        n_up, n_dn = count_up_down(i, n_orbitals)
        if n_up == N_up and n_dn == N_down:
            valid_indices.append(i)

if N_up is not None:
    print(f"Number of states in sector N_up={N_up}, N_down={N_down}: {len(valid_indices)}")            
    H_sector = H_sparse[np.ix_(valid_indices, valid_indices)]
    e_sector = eigsh(H_sector, k=1, which='SA', return_eigenvectors=False)[0]
    print("Sector GS:", e_sector)



/home/detlef/anaconda3/envs/NetKet/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


∣NK⟩ Tip: Prefer the new nk.driver.VMC_SR over VMC which supports minSR and SPRING.

/home/detlef/anaconda3/envs/NetKet/lib/python3.12/site-packages/netket/operator/_fermion2nd/jax.py:557: UserWarning: 
Consider using `netket.experimental.operator.ParticleNumberAndSpinConservingFermioperator2nd` to reduce the number of connected elements and
considerably reduce the computational cost.
You can convert this operator by calling `netket.experimental.operator.ParticleNumberAndSpinConservingFermioperator2nd.from_fermionoperator2nd`.

  super()._setup(self)


Ground state energy: -1.320234958271931
Lowest eigenvalues: [-3.20775094] N_up, N_down not supported, without particle number conservation
Number of states in sector N_up=2, N_down=2: 36
Sector GS: -1.320234958271927


In [26]:
import pennylane as qml
import numpy as np

def qubit_op_to_linear_combination(qubit_op):
    coeffs = []
    ops = []

    for term, coeff in qubit_op.terms.items():
        coeffs.append(float(np.real(coeff)))

        if len(term) == 0:
            ops.append(qml.Identity(wires=[]))
            continue

        paulis = []
        for wire, pauli in term:
            if pauli == "X":
                paulis.append(qml.PauliX(wire))
            elif pauli == "Y":
                paulis.append(qml.PauliY(wire))
            elif pauli == "Z":
                paulis.append(qml.PauliZ(wire))
            else:
                raise ValueError(pauli)

        ops.append(qml.prod(*paulis))

    return qml.ops.LinearCombination(coeffs, ops)
LCU_operator = qubit_op_to_linear_combination(qubit_op)

In [27]:
import numpy as np

# Find the first integer i such that 2^i >= len(LCU_operator.terms()[0])
num_terms = len(LCU_operator.terms()[0])
i = int(np.ceil(np.log2(num_terms)))
print(f"Number of terms: {num_terms}")
print(f"First integer i such that 2^i >= {num_terms}: {i}")
print(f"2^{i} = {2**i}")
num_terms = len(LCU_operator.terms()[0])
control = [n_qubits + i for i in range(int(np.ceil(np.log2(num_terms))))]



Number of terms: 29
First integer i such that 2^i >= 29: 5
2^5 = 32


In [28]:
eigv.shape, H_sparse.shape
eig_state = np.zeros((2**n_qubits, 2**len(control)), dtype=complex)
eig_state[:,0:1] = eigv
eig_state = eig_state.flatten()


@qml.qnode(qml.device("default.qubit"))
def circuit(lcu, control):
    qml.StatePrep(eig_state, wires=range(n_qubits + len(control)))
    qml.PrepSelPrep(lcu, control)
    return qml.state()

state = circuit(LCU_operator, control)
print(f"State after LCU block-encoding:\n {state}")

print("probability of control 0 state:", np.sum(np.abs(state.reshape(2**n_qubits,-1)[:,0])**2))

res_state = state.reshape(2**n_qubits, -1)[:, 0].copy()
res_state /= np.linalg.norm(res_state)
overlap = np.abs(np.vdot(eigv.flatten(), res_state))**2
print("Overlap with ground state:", overlap)

State after LCU block-encoding:
 [-1.34594688e-33-2.66412619e-33j  1.51097661e-18-1.25136993e-18j
 -2.61106757e-18-1.04723989e-18j ... -5.95106723e-18-5.13823410e-18j
 -3.43461352e-18-3.08639755e-18j -4.05806408e-18-5.28495516e-18j]
probability of control 0 state: 0.006431041321077891
Overlap with ground state: 0.9999999999999989


In [29]:
overlap = np.abs(np.vdot(eigv.flatten(), res_state))**2
print("Overlap with ground state:", overlap)
print("probability of control 0 state:", np.sum(np.abs(state.reshape(2**n_qubits,-1)[:,0])**2))


Overlap with ground state: 0.9999999999999989
probability of control 0 state: 0.006431041321077891


In [30]:
from openfermion import QubitOperator

beta = 0.1  # dein Faktor

# 1. Identity-Term
new_op = QubitOperator('', 1.0)  # '' steht für Identity

# 2. qubit_op multiplizieren und subtrahieren
for term, coeff in qubit_op.terms.items():
    new_op += QubitOperator(term, -beta * coeff)


In [31]:
new_op

(0.19999999999999996+0j) [] +
(-0.05+0j) [X0 Z1 X2] +
(-0.05+0j) [X0 Z1 Z2 Z3 X4] +
(-0.05+0j) [Y0 Z1 Y2] +
(-0.05+0j) [Y0 Z1 Z2 Z3 Y4] +
(0.2+0j) [Z0] +
(-0.2+0j) [Z0 Z1] +
(-0.05+0j) [X1 Z2 X3] +
(-0.05+0j) [X1 Z2 Z3 Z4 X5] +
(-0.05+0j) [Y1 Z2 Y3] +
(-0.05+0j) [Y1 Z2 Z3 Z4 Y5] +
(0.2+0j) [Z1] +
(-0.05+0j) [X2 Z3 Z4 Z5 X6] +
(-0.05+0j) [Y2 Z3 Z4 Z5 Y6] +
(0.2+0j) [Z2] +
(-0.2+0j) [Z2 Z3] +
(-0.05+0j) [X3 Z4 Z5 Z6 X7] +
(-0.05+0j) [Y3 Z4 Z5 Z6 Y7] +
(0.2+0j) [Z3] +
(-0.05+0j) [X4 Z5 X6] +
(-0.05+0j) [Y4 Z5 Y6] +
(0.2+0j) [Z4] +
(-0.2+0j) [Z4 Z5] +
(-0.05+0j) [X5 Z6 X7] +
(-0.05+0j) [Y5 Z6 Y7] +
(0.2+0j) [Z5] +
(0.2+0j) [Z6] +
(-0.2+0j) [Z6 Z7] +
(0.2+0j) [Z7]

In [45]:
def reflection_zero_state(ancilla_wires):
    k = len(ancilla_wires)
    qml.PauliX(ancilla_wires[-1])
    # global phase flip on |11...1>
    qml.ctrl(
        qml.PhaseShift,
        control=ancilla_wires[:-1],
        control_values=[0]*(k-1)
    )(np.pi, ancilla_wires[-1])
    qml.PauliX(ancilla_wires[-1])

    qml.GlobalPhase(np.pi) # debugging to easier read it



@qml.qnode(dev)
def apply_reflection(state_index):
    qml.BasisState(np.array(list(np.binary_repr(state_index, 2)), dtype=int), wires=[0,1])
    reflection_zero_state([0,1])
    return qml.state()

for i in range(4):
    out = apply_reflection(i)
    print(f"Input |{i:02b}> → Output amplitudes:", out.real)


Input |00> → Output amplitudes: [1. 0. 0. 0.]
Input |01> → Output amplitudes: [ 0. -1.  0.  0.]
Input |10> → Output amplitudes: [ 0.  0. -1.  0.]
Input |11> → Output amplitudes: [ 0.  0.  0. -1.]


In [46]:
@qml.qnode(qml.device("default.qubit"))
def apply_reflection():
    qml.StatePrep(state, wires=range(n_qubits + len(control)))
    reflection_zero_state(control)
    return qml.state()

state_reflect = apply_reflection()
    


In [47]:
res_state = state_reflect.reshape(2**n_qubits, -1)[:, 0]
np.linalg.norm(res_state)

np.float64(0.08019377358048374)

In [48]:
res_state /= np.linalg.norm(res_state)
overlap = np.abs(np.vdot(eigv.flatten(), res_state))**2
print("Overlap with ground state after reflection:", overlap)

Overlap with ground state after reflection: 0.9999999999999989


In [58]:
def OAA_step(lcu, ancilla_wires):
    qml.PrepSelPrep(lcu, ancilla_wires)
    reflection_zero_state(ancilla_wires)
    qml.adjoint(qml.PrepSelPrep(lcu, ancilla_wires))  # U†
    reflection_zero_state(ancilla_wires)


In [77]:
@qml.qnode(qml.device("default.qubit"))
def apply_oaa(k=3):
    qml.StatePrep(state, wires=range(n_qubits + len(control)))
    for _ in range(k):
        OAA_step(LCU_operator, control)
    return qml.state()

state_oaa = apply_oaa(10)
    


In [78]:
res_state = state_oaa.reshape(2**n_qubits, -1)[:, 0]
np.linalg.norm(res_state)

np.float64(0.9989661112173878)

In [79]:
res_state /= np.linalg.norm(res_state)
overlap = np.abs(np.vdot(eigv.flatten(), res_state))**2
print("Overlap with ground state after reflection:", overlap)

Overlap with ground state after reflection: 0.9999999999999996
